#  CREATE FLAG PARAMETER

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')


# CREATING DIMENSION MODEL

In [0]:
df_src = spark.sql('''
                   SELECT DISTINCT(Dealer_ID) AS Dealer_ID , DealerName
                   FROM parquet.`abfss://silver@carstorageaccount.dfr.azuredatalakestore.net/carsales`
                   ''')

In [0]:
display(df_src)

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_dealer'):
    df_sink = spark.sql('''
                   SELECT dim_dealer_key, Dealer_ID , DealerName
                   FROM cars_catalog.gold.dim_dealer
                   ''')
else:
    df_sink = spark.sql('''
                   SELECT 1 AS dim_dealer_key, Dealer_ID , DealerName
                   FROM parquet.`abfss://silver@carstorageaccount.dfr.azuredatalakestore.net/carsales`
                   WHERE 1=0
                   ''')

In [0]:
display(df_sink)

# Filtering old and new records

In [0]:
df_filter = df.src.join(df_sink,df_src['Dealer_ID'] == df_sink['Dealer_ID'],'left').select(df_src['Dealer_ID'],df_src['DealerName'],df_sink['dim_dealer_key'])


In [0]:
df_filter_old = df_filter.filter(col('dim_dealer_key').isNotNull())


In [0]:
df_filter_new = df_filter.filter(col('dim_dealer_key').isNull()).select(df_src['Dealer_ID'],df_src['DealerName'])

# Create Surrogate Key

In [0]:
if incremental_flag = '0':
    max_value = 1
else:
    max_value_df = spark.sql('''
                   SELECT MAX(dim_dealer_key) 
                   FROM cars_catalog.gold.dim_dealer
                   ''')
    max_value = max_value_df.collect()[0][0]+1

In [0]:
df_filter_new = df_filter_new.withColumn('dim_dealer_key',max_value + monotonically_increasing_id()) 

In [0]:
display(df_filter_new)


# Create Final df = df_filter_old + df_filter_new

In [0]:
df_final = df_filter_new.union(df_filter_old)

In [0]:
display(df_final)

# SCD TYPE - 1

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("cars_catalog.gold.dim_dealer"):
    delta_tbl = DeltaTable.forPath(spark, "abfss://gold@storagedatabricks.dfs.core.windows.net/dim_dealer")
    delta_tbl.alias("t").merge(df_final.alias("s"), "t.dim_dealer_key = s.dim_dealer_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_final.write.format("delta")\
        .mode("overwrite")\
        .option("path","abfss://gold@storagedatabricks.dfs.core.windows.net/dim_dealer")\
        .saveAsTable("cars_catalog.gold.dim_dealer")
DeltaTable.forName(spark


  File <command-5987932740977879>, line 3
    else:
    ^
IndentationError: expected an indented block after 'if' statement on line 1
